# v17: 
* feature nan droplama eklendi.
* time series kaldırıldı.
* antoninanın target hesaplaması var.


# TODO:
 * nan degerler nasıl halledilecek bak
 * time series interpolasyon ?
 * cv foldlar arası std düşecek
 * qwk optimizasyon https://www.kaggle.com/code/carlolepelaars/efficientnetb5-with-keras-aptos-2019#Metric-(Quadratic-Weighted-Kappa)-
 * seed blend
 * catboost denenecek

## TIME SERIES FEATURELARI ICIN:

* https://www.kaggle.com/code/antoninadolgorukova/cmi-piu-actigraphy-data-eda/notebook
* https://www.kaggle.com/code/ambrosm/piu-eda-which-makes-sense


In [1]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone, BaseEstimator, RegressorMixin
from sklearn.metrics import cohen_kappa_score, accuracy_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from pathlib import Path

from functools import partial
import scipy as sp


from sklearn.preprocessing import StandardScaler, OrdinalEncoder
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor

from scipy.stats import mode


from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
warnings.filterwarnings('ignore')


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# import random
# def seed_everything(seed):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = True
# seed_everything(2024)

## 1. DATA READING

In [3]:
train_df = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test_df = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

train_df.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,Fall,5.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,NaN,Fall,2.340,Fall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,5.0,7.0,33.0,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,5.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,6.0,9.0,37.0,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.22033,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,NaN,NaN,Winter,2.451,Summer,4.0,2.0,4.0,0.0,5.0,1.0,0.0,3.0,2.0,2.0,3.0,0.0,3.0,0.0,0.0,3.0,4.0,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
train_df.shape

(3960, 82)

## 2. SOME UTILITIES

In [5]:
class CFG:
    SEED_LIST = [42]
    N_SPLITS = 5
    target_col = "recalc_sii"

    lgb_params = {
    'objective'       : 'l2',
    'verbosity'       : -1,
    # 'n_iter'          : 200,
    'learning_rate'   : 0.025,
    'max_depth'       : 5,
    'num_leaves'      : 207,
    'min_data_in_leaf': 150,
    'lambda_l2'       : 5,
    'lambda_l1'       : 2,
    'bagging_freq'    : 7,
    }

    catboost_params = {
        'learning_rate': 0.035,
        'depth': 5,
        # 'iterations': 200,
        'verbose': False,
        'l2_leaf_reg': 5, 
        'border_count': 254,
        # 'random_strength': 2.5,
        'task_type': 'CPU',
    }

### 2.1. RECALCULATING THE TARGET

Target variable has some faults because of the survey results.

In [6]:
train_cols = set(train_df.columns)
test_cols = set(test_df.columns)
columns_not_in_test = sorted(list(train_cols - test_cols))

train_with_sii = train_df[train_df['sii'].notna()][columns_not_in_test]
train_with_sii[train_with_sii.isna().any(axis=1)].head().style.applymap(
    lambda x: 'background-color: #FFC0CB' if pd.isna(x) else ''
)

PCIAT_cols = [f'PCIAT-PCIAT_{i+1:02d}' for i in range(20)]
recalc_total_score = train_with_sii[PCIAT_cols].sum(
    axis=1, skipna=True
)
(recalc_total_score == train_with_sii['PCIAT-PCIAT_Total']).all()


def recalculate_sii(row):
    if pd.isna(row['PCIAT-PCIAT_Total']):
        return np.nan
    max_possible = row['PCIAT-PCIAT_Total'] + row[PCIAT_cols].isna().sum() * 5
    if row['PCIAT-PCIAT_Total'] <= 30 and max_possible <= 30:
        return 0
    elif 31 <= row['PCIAT-PCIAT_Total'] <= 49 and max_possible <= 49:
        return 1
    elif 50 <= row['PCIAT-PCIAT_Total'] <= 79 and max_possible <= 79:
        return 2
    elif row['PCIAT-PCIAT_Total'] >= 80 and max_possible >= 80:
        return 3
    return np.nan

train_df['recalc_sii'] = train_df.apply(recalculate_sii, axis=1)

mismatch_rows = train_df[
    (train_df['recalc_sii'] != train_df['sii']) & train_df['sii'].notna()
]

mismatch_rows[PCIAT_cols + [
    'PCIAT-PCIAT_Total', 'sii', 'recalc_sii'
]].style.applymap(
    lambda x: 'background-color: #FFC0CB' if pd.isna(x) else ''
)

,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,sii,recalc_sii
24,2.000000,2.000000,3.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,nan,2.000000,30.000000,0.000000,nan
93,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000000,0.000000,nan
104,5.000000,2.000000,4.000000,2.000000,nan,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,45.000000,1.000000,nan
141,1.000000,2.000000,4.000000,2.000000,2.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,3.000000,0.000000,nan,0.000000,2.000000,0.000000,26.000000,0.000000,nan
142,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,3.000000,0.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,nan,1.000000,26.000000,0.000000,nan
270,3.000000,3.000000,4.000000,2.000000,4.000000,2.000000,1.000000,3.000000,2.000000,2.000000,4.000000,0.000000,2.000000,1.000000,4.000000,nan,2.000000,3.000000,4.000000,2.000000,48.000000,1.000000,nan
368,2.000000,3.000000,4.000000,2.000000,5.000000,1.000000,2.000000,nan,nan,nan,2.000000,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,1.000000,nan,nan,31.000000,1.000000,nan
592,3.000000,0.000000,3.000000,0.000000,3.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2.000000,0.000000,1.000000,nan,nan,1.000000,2.000000,1.000000,1.000000,0.000000,21.000000,0.000000,nan
724,3.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000,nan,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,3.000000,0.000000,1.000000,29.000000,0.000000,nan
877,5.000000,5.000000,5.000000,4.000000,5.000000,0.000000,5.000000,5.000000,5.000000,5.000000,4.000000,nan,4.000000,5.000000,5.000000,1.000000,5.000000,0.000000,5.000000,5.000000,78.000000,2.000000,nan


### 2.2. ENCODING TIME SERIES DATASET

Creating features based on the time series data to potentially boost the performance.

In [7]:
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    
    return df.describe().values.reshape(-1), filename.split('=')[1]


def load_time_series(dirname):
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    
    return df

In [8]:
ts_df = pd.read_parquet('/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=5f9dddb4/part-0.parquet')
ts_df.head()

,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
0,0,0.697604,0.663398,0.088047,0.030687,4.241837,0.0,28.500000,4112.000000,52560000000000,1,1,47.0
1,1,0.553945,0.590495,0.346914,0.028160,22.999907,0.0,40.500000,4108.583496,52565000000000,1,1,47.0
2,2,-0.093451,0.641497,0.469141,0.059117,31.973183,0.0,40.500000,4105.166504,52570000000000,1,1,47.0
3,3,-0.387135,-0.353438,0.573372,0.046419,40.968136,0.0,40.500000,4101.750000,52575000000000,1,1,47.0
4,4,-0.343724,-0.355339,0.792214,0.067833,57.952835,0.0,33.666668,4098.333496,52580000000000,1,1,47.0


In [9]:
ts_df.isnull().sum()

step                   0
X                      0
Y                      0
Z                      0
enmo                   0
anglez                 0
non-wear_flag          0
light                  0
battery_voltage        0
time_of_day            0
weekday                0
quarter                0
relative_date_PCIAT    0
dtype: int64

## 3. QWK METRIC

In [10]:
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

## 4. SOME FEATURE ENGINEERING

In [11]:
bins = np.arange(0, 101, 10)
labels = [
    "1-10: Needs constant supervision (24 hour care)",
    "11-20: Needs considerable supervision",
    "21-30: Unable to function in almost all areas",
    "31-40: Major impairment in functioning in several areas",
    "41-50: Moderate degree of interference in functioning",
    "51-60: Variable functioning with sporadic difficulties",
    "61-70: Some difficulty in a single area",
    "71-80: No more than slight impairment in functioning",
    "81-90: Good functioning in all areas",
    "91-100: Superior functioning"
]

physical_features = ['Physical-Season',
 'Physical-BMI',
 'Physical-Height',
 'Physical-Weight',
 'Physical-Waist_Circumference',
 'Physical-Diastolic_BP',
 'Physical-HeartRate',
 'Physical-Systolic_BP']

num_cols = [
    col for col in train_df.select_dtypes(include=['number']).columns 
    if not col.startswith('PCIAT-') and col not in ['sii', 'recalc_sii']
]

physical_cols = [col for col in physical_features if col in num_cols]
bp_hr_cols = [
    'Physical-Diastolic_BP', 'Physical-Systolic_BP',
    'Physical-HeartRate'
]

lbs_to_kg = 0.453592
inches_to_cm = 2.54

def feature_engineering(data):

    # data[physical_cols] = data[physical_cols].replace(0, np.nan)
    # data.loc[data['Physical-Systolic_BP'] <= data['Physical-Diastolic_BP'], bp_hr_cols] = np.nan
    data.loc[data['CGAS-CGAS_Score'] == 999, 'CGAS-CGAS_Score'] = np.nan

    data['Physical-Weight_recalc'] = data['Physical-Weight'] * lbs_to_kg
    data['Physical-Height_recalc'] = data['Physical-Height'] * inches_to_cm
    data['Physical-Waist_Circumference_recalc'] = data['Physical-Waist_Circumference'] * inches_to_cm
    
    data['Physical-BMI_recalc'] = np.where(
        data['Physical-Weight_recalc'].notna() & data['Physical-Height_recalc'].notna(),
        data['Physical-Weight_recalc'] / ((data['Physical-Height_recalc'] / 100) ** 2),
        np.nan  # If either is NaN, set BMI to NaN
    )
    
    conditions = [
        data['Physical-BMI_recalc'] < 18.5,
        (data['Physical-BMI_recalc'] >= 18.5) & (data['Physical-BMI_recalc'] <= 24.9),
        (data['Physical-BMI_recalc'] >= 25) & (data['Physical-BMI_recalc'] <= 29.9),
        data['Physical-BMI_recalc'] >= 30
    ]
    categories = ['Underweight', 'Normal weight', 'Overweight', 'Obesity']
    data['bmi_category'] = np.select(conditions, categories, default='Unknown')

    

    data['bp_ratio'] = data['Physical-Systolic_BP'] / data['Physical-Diastolic_BP']
    data['height_waist_ratio'] = data['Physical-Height'] / data['Physical-Waist_Circumference']
    data['cardio_index'] = (data['Physical-HeartRate'] * data['Physical-Systolic_BP']) / 1000
    
    data['total_fitness_duration'] = (data['Fitness_Endurance-Time_Mins'] * 60 + 
                                      data['Fitness_Endurance-Time_Sec'])
    
    data['endurance_efficiency'] = (data['Fitness_Endurance-Max_Stage'] / 
                                    data['total_fitness_duration'].replace(0, 1))
    
    data['fat_muscle_ratio'] = data['BIA-BIA_Fat'] / (data['BIA-BIA_SMM'] + 1e-5)
    data['water_balance'] = data['BIA-BIA_ECW'] / (data['BIA-BIA_ICW'] + 1e-5)


    data['fat_tbw_ratio'] = data['BIA-BIA_Fat'] / data['BIA-BIA_TBW'] # 
    data['fat_lst_ratio'] = data['BIA-BIA_Fat'] / data['BIA-BIA_LST'] # 
    data['bmi_hr'] = data['Physical-BMI'] * data['Physical-HeartRate'] # 
    # data['bmi_systolic'] = data['Physical-BMI'] * data['Physical-Systolic_BP'] # 
    # data['bmi_diastolic'] = data['Physical-BMI'] * data['Physical-Diastolic_BP'] # 
    # data['stamina_index'] = data['Fitness_Endurance-Time_Sec'] / (data['Fitness_Endurance-Max_Stage'] + 1e-5) # 
    # data['age_normalized_endurance'] = data['Fitness_Endurance-Time_Mins'] / (data['Basic_Demos-Age'] + 1e-5) # 
    # data['sds_cgas_ratio'] = data['SDS-SDS_Total_T'] / (data['CGAS-CGAS_Score'] + 1e-5) # 
    # data['sds_raw_cgas_ratio'] = data['SDS-SDS_Total_Raw'] / (data['CGAS-CGAS_Score'] + 1e-5) #
    # data['ponderal_index'] = data['Physical-Weight'] / (data['Physical-Height']**3 + 1e-5) #


    data['muscle_height_ratio'] = data['BIA-BIA_SMM'] / data['Physical-Height']
    data['bmi_activity'] = data['Physical-BMI'] * data['BIA-BIA_Activity_Level_num']
    data['hr_fitness'] = data['Physical-HeartRate'] * data['Fitness_Endurance-Max_Stage']

    data['internet_hours_per_age'] = (data['PreInt_EduHx-computerinternet_hoursday'] / 
                                      data['Basic_Demos-Age'])
    data['internet_activity_ratio'] = (
        data['PreInt_EduHx-computerinternet_hoursday'] / 
        (data['PAQ_A-PAQ_A_Total'].fillna(0) + data['PAQ_C-PAQ_C_Total'].fillna(0) + 1)
    )

    data['age_adj_bmi'] = data['Physical-BMI'] / np.log1p(data['Basic_Demos-Age'])
    
    data['age_group'] = pd.cut(data['Basic_Demos-Age'], 
                                bins=[0, 8, 12, 16, 22], 
                                labels=['young_child', 'child', 'teen', 'young_adult'])
    
    data['screen_activity'] = (
        data['PreInt_EduHx-computerinternet_hoursday'] * 
        (data['PAQ_A-PAQ_A_Total'].fillna(0) + data['PAQ_C-PAQ_C_Total'].fillna(0))
    )

    data['CGAS_Score_Bin'] = pd.cut(
            data['CGAS-CGAS_Score'], bins=bins, labels=labels
        )


    
    features_to_drop = [
        'Physical-Weight_recalc',
        'Physical-Height_recalc',
        'Physical-Waist_Circumference_recalc',
        'Physical-BMI_recalc'
    ]
    
    data.drop(columns=features_to_drop, inplace=True)

    return data


train_df = feature_engineering(train_df)
test_df = feature_engineering(test_df)

In [12]:
train_df.shape

(3960, 103)

In [13]:
train_df.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,recalc_sii,bmi_category,bp_ratio,height_waist_ratio,cardio_index,total_fitness_duration,endurance_efficiency,fat_muscle_ratio,water_balance,fat_tbw_ratio,fat_lst_ratio,bmi_hr,muscle_height_ratio,bmi_activity,hr_fitness,internet_hours_per_age,internet_activity_ratio,age_adj_bmi,age_group,screen_activity,CGAS_Score_Bin
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,Fall,5.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0,2.0,Underweight,NaN,NaN,NaN,NaN,NaN,0.471502,0.337876,0.281845,0.236750,NaN,0.424811,33.754631,NaN,0.6,3.000000,9.419409,young_child,0.00,51-60: Variable functioning with sporadic diff...
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,NaN,Fall,2.340,Fall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0,0.0,Underweight,1.626667,2.181818,8.540,NaN,NaN,0.257668,0.286183,0.146768,0.100656,982.491320,0.321056,28.071181,NaN,0.0,0.000000,6.095579,child,0.00,NaN
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,5.0,7.0,33.0,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,5.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0,0.0,Underweight,1.800000,NaN,10.998,453.0,0.011038,NaN,NaN,NaN,NaN,1564.977430,NaN,NaN,470.0,0.2,0.630915,6.943046,child,4.34,71-80: No more than slight impairment in funct...
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,6.0,9.0,37.0,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.22033,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,NaN,NaN,Winter,2.451,Summer,4.0,2.0,4.0,0.0,5.0,1.0,0.0,3.0,2.0,2.0,3.0,0.0,3.0,0.0,0.0,3.0,4.0,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0,1.0,Underweight,1.950000,NaN,11.349,577.0,0.010399,0.710893,0.512842,0.409254,0.319414,1774.357653,0.472854,54.877041,582.0,0.0,0.000000,7.944265,child,0.00,71-80: No more than slight impairment in funct...
4,0016bb22,Spring,18,1,Summer,Na

In [14]:
train_df['Basic_Demos-Age'].value_counts()

Basic_Demos-Age
8     490
9     467
7     436
10    420
6     369
11    334
12    291
13    236
14    200
15    178
16    151
17    114
5     112
18     74
21     29
19     27
20     24
22      8
Name: count, dtype: int64

## 4.1. FILTERING COLUMNS WITH LOTS OF MISSING DATA 

In [15]:
# Calculate the percentage of missing values for each column
missing_percentage = train_df.isnull().mean()

# Filter columns with more than 30% NaN values
columns_with_nans = missing_percentage[missing_percentage > 0.5]

columns_to_drop = [col for col in columns_with_nans.index if col not in ['recalc_sii']]
# columns_to_drop.append('id') 
len(columns_to_drop)

19

In [16]:
train_df = train_df.dropna(subset=[CFG.target_col])

In [17]:
train_df = train_df.drop(columns=columns_to_drop)
train_df.shape

(2719, 84)

In [18]:
# ts_train = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
# ts_test = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

# time_series_cols = ts_train.columns.tolist()
# time_series_cols.remove("id")

# train_df = pd.merge(train_df, ts_train, how="left", on='id')
# test_df = pd.merge(test_df, ts_test, how="left", on='id')

train_df = train_df.set_index('id')
test_df = test_df.set_index('id')

In [19]:
train_df.shape

(2719, 83)

In [20]:
num_cols = [
    col for col in train_df.select_dtypes(include=['number']).columns 
    if not col.startswith('PCIAT-') and col not in ['sii', 'recalc_sii']
]

# Create cat_cols for object and category columns
cat_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
cat_cols.remove('PCIAT-Season')

# Display the lists
print("Numerical Columns:", num_cols)
print("Categorical Columns:", cat_cols)


print(f"Number of features in num_cols: {len(num_cols)}")
print(f"Number of features in cat_cols: {len(cat_cols)}")


feature_cols = num_cols + cat_cols
print(f"Number of features in feature_cols: {len(feature_cols)}")


Numerical Columns: ['Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Height', 'Physical-Weight', 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-computerinternet_hoursday', 'bp_ratio', 'cardio_index', 'fat_muscle_ratio', 'water_balance', 'fat_tbw_ratio', 'fat_lst_ratio', 'bmi_hr', 'internet_hours_per_age', 'internet_activity_ratio', 'age_adj_bmi', 'screen_activity']
Categorical Columns: ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'FGC

In [21]:
imputer = SimpleImputer(
    strategy='mean',
)

train_df[num_cols] = imputer.fit_transform(train_df[num_cols])
test_df[num_cols] = imputer.transform(test_df[num_cols])

encoder = OrdinalEncoder(
    dtype=np.int32,
    handle_unknown='use_encoded_value',
    unknown_value=-1,
    encoded_missing_value=-2,
)

train_df[cat_cols] = encoder.fit_transform(train_df[cat_cols])
train_df[cat_cols] = train_df[cat_cols].astype('category')

test_df[cat_cols] = encoder.transform(test_df[cat_cols])
test_df[cat_cols] = test_df[cat_cols].astype('category')

In [22]:
train_df.head()

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,recalc_sii,bmi_category,bp_ratio,cardio_index,fat_muscle_ratio,water_balance,fat_tbw_ratio,fat_lst_ratio,bmi_hr,internet_hours_per_age,internet_activity_ratio,age_adj_bmi,age_group,screen_activity,CGAS_Score_Bin
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00008ff9,0,5.0,0.0,3,51.000000,0,16.877316,46.0,50.8,69.780032,81.841424,117.134334,0,0.0,0.0,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,0,2.000000,2.668550,16.879200,932.498000,1492.000000,8.255980,41.586200,13.81770,3.061430,9.213770,1.000000,24.434900,8.895360,38.917700,19.541300,32.690900,Fall,5.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,55.0,-2,40.987266,57.658702,0,3.0,2.0,2.0,3,1.713216,9.612776,0.471502,0.337876,0.281845,0.236750,1563.301379,0.6,3.000000,9.419409,3,0.00,3
000fd460,2,9.0,0.0,-2,65.145189,0,14.035590,48.0,46.0,75.000000,70.000000,122.000000,0,3.0,0.0,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,3,2.000000,2.579490,14.037100,936.656000,1498.650000,6.019930,42.029100,12.82540,1.211720,3.970850,1.000000,21.035200,14.974000,39.449700,15.410700,27.055200,Fall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,46.000000,64.000000,2,0.0,0.0,0.0,3,1.626667,8.540000,0.257668,0.286183,0.146768,0.100656,982.491320,0.0,0.000000,6.095579,0,0.00,-2
00105258,2,10.0,1.0,0,71.000000,0,16.648696,56.5,75.6,65.000000,94.000000,117.000000,0,20.0,1.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,-2,2.673141,6.960985,19.233414,1236.588338,2069.906637,20.780299,73.975922,14.99489,4.238525,15.772228,1.733629,33.132889,20.062741,67.014937,34.379683,53.913188,Fall,5.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,1.0,28.0,0,38.000000,54.000000,2,2.0,0.0,0.0,3,1.800000,10.998000,0.648436,0.566713,0.398129,0.313163,1564.977430,0.2,0.630915,6.943046,0,4.34,5
00115b9f,3,9.0,0.0,0,71.000000,2,18.292347,56.0,81.6,60.000000,97.000000,117.000000,2,18.0,1.0,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,2,3.000000,3.841910,18.294300,1131.430000,1923.440000,15.592500,62.775700,14.07400,4.220330,18.824300,2.000000,30.404100,16.779000,58.933800,26.479800,45.996600,Summer,4.0,2.0,4.0,0.0,5.0,1.0,0.0,3.0,2.0,2.0,3.0,0.0,3.0,0.0,0.0,3.0,4.0,3.0,4.0,1.0,44.0,2,31.000000,45.000000,3,0.0,1.0,1.0,3,1.950000,11.349000,0.710893,0.512842,0.409254,0.319414,1774.357653,0.0,0.000000,7.944265,0,0.00,5
001f3379,1,13.0,1.0,3,50.000000,2,22.279952,59.5,112.2,60.000000,73.000000,102.000000,2,12.0,0.0,6.0,0.0,10.0,1.0,11.0,1.0,8.0,0.0,2,2.000000,4.330360,30.186500,1330.970000,1996.450000,30.212400,84.028500,16.68770,13.498800,67.971500,2.000000,32.914100,20.902000,79.698200,35.380400,63.126500,Summer,3.0,3.0,3.0,0.0,2.0,1.0,0.0,2.0,2.0,1.0,0.0,1.0,3.0,3.0,2.0,1.0,3.0,1.0,2.0,1.0,34.0,2,40.000000,56.000000,1,0.0,1.0,1.0,0,1.700000,7.446000,1.921162,0.917916,1.076751,0.852861,1626.436495,0.0,0.000000,8.442390,1,0.00,2


## 5. TRAINING THE MODEL WITH MULTIPLE SEED CV SETUP TO IMPROVE GENERALIZATION

In [ ]:
from scipy.stats import ttest_rel
import random

def train_model_seeds(model_class, train_data, max_seeds=20):
    # This list will store results for seed 42 and accepted seeds
    all_seed_results = []
    accepted_seeds = []
    seed_fold_qwks = {}

    # Static seed for comparison
    fixed_seed = 12
    
    # Generate results for the fixed seed first
    seed_results_42 = run_model_with_seed(fixed_seed, model_class, train_data)
    seed_fold_qwks[fixed_seed] = seed_results_42['fold_qwks']
    all_seed_results.append(seed_results_42)

    print(f"Fixed seed ({fixed_seed}) results calculated.")

    while len(accepted_seeds) < max_seeds:
        # Generate a random seed
        new_seed = random.randint(1, 1e6)

        if new_seed in seed_fold_qwks:
            continue  # Avoid duplicate seed processing

        # Run model with the new seed
        new_seed_results = run_model_with_seed(new_seed, model_class, train_data)
        seed_fold_qwks[new_seed] = new_seed_results['fold_qwks']

        # Perform t-test between the fixed seed and the new seed
        t_stat, p_val = ttest_rel(seed_fold_qwks[fixed_seed], seed_fold_qwks[new_seed])

        print(f"Seed {new_seed} compared with fixed seed {fixed_seed}: T-stat = {t_stat:.4f}, P-value = {p_val:.4f}")

        # Check p-value threshold
        if p_val < 0.05:
            accepted_seeds.append(new_seed)
            all_seed_results.append(new_seed_results)
            print(f"Seed {new_seed} added to the accepted seeds list with T-stat = {t_stat:.4f}, P-value = {p_val:.4f}")

    print("\nFinal accepted seeds:", accepted_seeds)
    return all_seed_results, accepted_seeds

def run_model_with_seed(seed, model_class, train_data):
    X = train_data[feature_cols]
    y = train_data[CFG.target_col]

    SKF = StratifiedKFold(n_splits=CFG.N_SPLITS, shuffle=True, random_state=seed)

    fold_qwks = []
    fold_qwks_std = []
    oof_non_rounded = np.zeros(len(y), dtype=float)

    for fold, (train_idx, test_idx) in enumerate(SKF.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_val_pred = model.predict(X_val)
        y_val_pred_rounded = y_val_pred.round(0).astype(int)

        oof_non_rounded[test_idx] = y_val_pred
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        fold_qwks.append(val_kappa)

    mean_val_qwk = np.mean(fold_qwks)
    fold_qwks_std.append(np.std(fold_qwks))
    print(f"SEED {seed} QWK STD {fold_qwks_std}")

    KappaOptimizer = minimize(
        evaluate_predictions,
        x0=[0.5, 1.5, 2.5],
        args=(y, oof_non_rounded),
        method='Nelder-Mead'
    )
    assert KappaOptimizer.success, "Optimization did not converge."

    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOptimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    return {
        'seed': seed,
        'fold_qwks': fold_qwks,
        'fold_qwks_std': fold_qwks_std,
        'mean_val_qwk': mean_val_qwk,
        'optimized_qwk': tKappa,
        'optimized_thresholds': KappaOptimizer.x
    }


## 6. FINAL MODEL TRAINING WITH SELECTED SEEDS

In [24]:
def train_model(model_class, train_data, seeds=[12, 582484, 675412, 288380, 70743, 887692, 659334, 863144, 522781, 782194]):
    # This list will store results for each seed
    all_seed_results = []
    seed_fold_qwks = []

    for seed in seeds:
        X = train_data[feature_cols]
        y = train_data[CFG.target_col]

        SKF = StratifiedKFold(n_splits=CFG.N_SPLITS, shuffle=True, random_state=seed)

        train_S = []
        test_S = []

        oof_non_rounded = np.zeros(len(y), dtype=float)
        oof_rounded = np.zeros(len(y), dtype=int)

        models = []

        for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc=f"Training Folds [seed={seed}]", total=CFG.N_SPLITS)):
            X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

            model = clone(model_class)
            model.fit(X_train, y_train)

            models.append(model)

            y_train_pred = model.predict(X_train)
            y_val_pred = model.predict(X_val)

            oof_non_rounded[test_idx] = y_val_pred
            y_val_pred_rounded = y_val_pred.round(0).astype(int)
            oof_rounded[test_idx] = y_val_pred_rounded

            train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
            val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

            train_S.append(train_kappa)
            test_S.append(val_kappa)

            print(f"Seed {seed} - Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")

        mean_train_qwk = np.mean(train_S)
        mean_val_qwk = np.mean(test_S)

        print(f"Seed {seed}: Mean Train QWK --> {mean_train_qwk:.4f}")
        print(f"Seed {seed}: Mean Validation QWK ---> {mean_val_qwk:.4f}")

        KappaOPtimizer = minimize(
            evaluate_predictions,
            x0=[0.5, 1.5, 2.5],
            args=(y, oof_non_rounded),
            method='Nelder-Mead'
        )

        assert KappaOPtimizer.success, "Optimization did not converge."

        oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
        tKappa = quadratic_weighted_kappa(y, oof_tuned)

        print(f"Seed {seed} ----> || Optimized QWK SCORE :: {tKappa:.3f}")

        all_seed_results.append({
            'seed': seed,
            'mean_train_qwk': mean_train_qwk,
            'mean_val_qwk': mean_val_qwk,
            'optimized_thresholds': KappaOPtimizer.x,
            'optimized_qwk': tKappa,
            'models': models
        })

        seed_fold_qwks.append(test_S)

    # Print all seed results at the end
    print("\nSummary of all seeds:")
    for res in all_seed_results:
        print(f"Seed {res['seed']}: "
              f"Mean Train QWK = {res['mean_train_qwk']:.4f}, "
              f"Mean Validation QWK = {res['mean_val_qwk']:.4f}, "
              f"Optimized QWK = {res['optimized_qwk']:.3f}, "
              f"Optimized thresholds = {res['optimized_thresholds']}")

    num_seeds = len(seeds)
    for i in range(num_seeds):
        for j in range(i+1, num_seeds):
            t_stat, p_val = ttest_rel(seed_fold_qwks[i], seed_fold_qwks[j])
            print(f"T-test between seed {all_seed_results[i]['seed']} and {all_seed_results[j]['seed']}:")
            print(f"  T-statistic = {t_stat:.4f}, p-value = {p_val:.4f}")

    return all_seed_results

## 7. INFERENCE

In [25]:
def MakePredictions(test_data, models, thresholds):
    # Get predictions from each fold's model

    test_data = test_data[feature_cols]
    
    preds_per_model = []
    for model in models:
        preds = model.predict(test_data)
        preds_per_model.append(preds)
    
    # Average predictions from all models (ensemble)
    mean_preds = np.mean(preds_per_model, axis=0)

    # Apply the optimized thresholds to convert continuous predictions to discrete classes
    final_predictions = threshold_Rounder(mean_preds, thresholds)

    return final_predictions


In [26]:
# seed 12**
# * SEED 582484 STD 0.03029330159401383  P-value = 0.0411**
# * SEED 887692  STD 0.02788239994265859 P-value = 0.1070 **
# * SEED 789218 STD 0.03348157285317313 P-value = 0.1329
# * SEED 782194 STD 0.03340845336739705 P-value = 0.1266**
# * SEED 522781 STD 0.01948893714067069 P-value = 0.1245**
# * SEED 802413  STD 0.008356112092699325 P-value = 0.1595
# * SEED 675412 STD 0.01753320176596404 P-value = 0.0367 **
# * SEED 205323 STD 0.008644273627400204 P-value = 0.1875
# * SEED 363177 STD 0.020553443791899568 P-value = 0.1413
# * 288380**
# * 70743**
# * SEED 676127 STD 0.02778639548591492 P-value = 0.1667
# * SEED 863144 STD 0.009414582744977314 P-value = 0.1286**
# * SEED 659334 STD 0.024758027476594974 P-value = 0.1127 ** 

In [27]:
lgbm1 = LGBMRegressor(**CFG.lgb_params, random_state=12, verbose=2, n_estimators=200)
lgbm2 = LGBMRegressor(**CFG.lgb_params, random_state=22, verbose=2, n_estimators=200)
lgbm3 = LGBMRegressor(**CFG.lgb_params, random_state=32, verbose=2, n_estimators=200)
lgbm4 = LGBMRegressor(**CFG.lgb_params, random_state=42, verbose=2, n_estimators=200)
lgbm5 = LGBMRegressor(**CFG.lgb_params, random_state=52, verbose=2, n_estimators=200)
lgbm6 = LGBMRegressor(**CFG.lgb_params, random_state=62, verbose=2, n_estimators=200)
lgbm7 = LGBMRegressor(**CFG.lgb_params, random_state=72, verbose=2, n_estimators=200)
lgbm8 = LGBMRegressor(**CFG.lgb_params, random_state=82, verbose=2, n_estimators=200)
lgbm9 = LGBMRegressor(**CFG.lgb_params, random_state=92, verbose=2, n_estimators=200)
lgbm10 = LGBMRegressor(**CFG.lgb_params, random_state=102, verbose=2, n_estimators=200)


# Identify categorical features
categorical_features = [train_df[feature_cols].columns.get_loc(col) for col in train_df[feature_cols].select_dtypes(exclude=['number'])]
# categorical_indices = [train_df.columns.get_loc(col) for col in categorical_features]


cat1 = CatBoostRegressor(**CFG.catboost_params, random_seed=12, iterations=200, cat_features=categorical_features)
cat2 = CatBoostRegressor(**CFG.catboost_params, random_seed=22, iterations=200, cat_features=categorical_features)
cat3 = CatBoostRegressor(**CFG.catboost_params, random_seed=32, iterations=200, cat_features=categorical_features)
cat4 = CatBoostRegressor(**CFG.catboost_params, random_seed=42, iterations=200, cat_features=categorical_features)
cat5 = CatBoostRegressor(**CFG.catboost_params, random_seed=52, iterations=200, cat_features=categorical_features)
cat6 = CatBoostRegressor(**CFG.catboost_params, random_seed=62, iterations=200, cat_features=categorical_features)
cat7 = CatBoostRegressor(**CFG.catboost_params, random_seed=72, iterations=200, cat_features=categorical_features)
cat8 = CatBoostRegressor(**CFG.catboost_params, random_seed=82, iterations=200, cat_features=categorical_features)
cat9 = CatBoostRegressor(**CFG.catboost_params, random_seed=92, iterations=200, cat_features=categorical_features)
cat10 = CatBoostRegressor(**CFG.catboost_params, random_seed=102, iterations=200, cat_features=categorical_features)

model_lgb = VotingRegressor([
    ('lgb_1', lgbm1),
    ('lgb_2', lgbm2),
    ('lgb_3', lgbm3),
    ('lgb_4', lgbm4),
    ('lgb_5', lgbm5),
    ('lgb_6', lgbm6),
    ('lgb_7', lgbm7),
    ('lgb_8', lgbm8),
    ('lgb_9', lgbm9),
    ('lgb_10', lgbm10),
])

model_cb = VotingRegressor([
    ('cat1', cat1),
    ('cat2', cat2),
    ('cat3', cat3),
    ('cat4', cat4),
    ('cat5', cat5),
    ('cat6', cat6),
    ('cat7', cat7),
    ('cat8', cat8),
    ('cat9', cat9),
    ('cat10', cat10)
]) 


model = VotingRegressor(estimators=[
    ('lgb', model_lgb),
    ('cb', model_cb),
], weights=[0.9, 0.1])

all_seed_results = train_model(model, train_df)

result1 = all_seed_results[0]
result2 = all_seed_results[1]
result3 = all_seed_results[2]
result4 = all_seed_results[3]
result5 = all_seed_results[4]
result6 = all_seed_results[5]
result7 = all_seed_results[6]
result8 = all_seed_results[7]
result9 = all_seed_results[8]
result10 = all_seed_results[9]


thresholds1 = result1['optimized_thresholds']
models1 = result1['models']

thresholds2 = result2['optimized_thresholds']
models2 = result2['models']

thresholds3 = result3['optimized_thresholds']
models3 = result3['models']

thresholds4 = result4['optimized_thresholds']
models4 = result4['models']

thresholds5 = result5['optimized_thresholds']
models5 = result5['models'] 

thresholds6 = result6['optimized_thresholds']
models6 = result6['models']

thresholds7 = result7['optimized_thresholds']
models7 = result7['models']

thresholds8 = result8['optimized_thresholds']
models8 = result8['models']

thresholds9 = result9['optimized_thresholds']
models9 = result9['models'] 

thresholds10 = result10['optimized_thresholds']
models10 = result10['models']

test_preds1 = MakePredictions(test_df, models1, thresholds1)
test_preds2 = MakePredictions(test_df, models2, thresholds2)
test_preds3 = MakePredictions(test_df, models3, thresholds3)
test_preds4 = MakePredictions(test_df, models4, thresholds4)
test_preds5 = MakePredictions(test_df, models5, thresholds5)
test_preds6 = MakePredictions(test_df, models6, thresholds6)
test_preds7 = MakePredictions(test_df, models7, thresholds7)
test_preds8 = MakePredictions(test_df, models8, thresholds8)
test_preds9 = MakePredictions(test_df, models9, thresholds9)
test_preds10 = MakePredictions(test_df, models10, thresholds10)


all_preds = np.vstack([test_preds1, test_preds2, test_preds3, test_preds4, test_preds5, test_preds6, test_preds7, test_preds8, test_preds9, test_preds10])  # Ensure 2D structure
final_preds = mode(all_preds, axis=0).mode.flatten()

# final_preds = (test_preds1 + test_preds2 + test_preds3 + test_preds4 + test_preds5 + test_preds6) / 6

# final_preds = final_preds.round(0).astype(int)
print("test preds 1:", test_preds1)
print("test preds 2:", test_preds2)
print("test preds 3:", test_preds3)
print("test preds 4:", test_preds4)
print("test preds 5:", test_preds5)
print("test preds 6:", test_preds6)
print("test preds 7:", test_preds7)
print("test preds 8:", test_preds8)
print("test preds 9:", test_preds9)
print("test preds 10:", test_preds10)

print("Test Predictions:", final_preds)

Training Folds [seed=12]:  20%|██        | 1/5 [00:16<01:05, 16.47s/it]

Seed 12 - Fold 1 - Train QWK: 0.4733, Validation QWK: 0.3763


Training Folds [seed=12]:  40%|████      | 2/5 [00:32<00:48, 16.09s/it]

Seed 12 - Fold 2 - Train QWK: 0.4787, Validation QWK: 0.3521


Training Folds [seed=12]:  60%|██████    | 3/5 [00:48<00:32, 16.13s/it]

...

Training Folds [seed=782194]:  80%|████████  | 4/5 [01:03<00:16, 16.04s/it]

Seed 782194 - Fold 4 - Train QWK: 0.4809, Validation QWK: 0.2923


Training Folds [seed=782194]: 100%|██████████| 5/5 [01:19<00:00, 15.96s/it]

Seed 782194 - Fold 5 - Train QWK: 0.4544, Validation QWK: 0.3738
Seed 782194: Mean Train QWK --> 0.4728
Seed 782194: Mean Validation QWK ---> 0.3562


Seed 782194 ----> || Optimized QWK SCORE :: 0.465

Summary of all seeds:
Seed 12: Mean Train QWK = 0.4725, Mean Validation QWK = 0.3741, Optimized QWK = 0.477, Optimized thresholds = [0.59339563 0.96007468 2.76467637]
Seed 582484: Mean Train QWK = 0.4748, Mean Validation QWK = 0.3843, Optimized QWK = 0.470, Optimized thresholds = [0.48578915 1.00704188 2.88473556]
Seed 675412: Mean Train QWK = 0.4771, Mean Validation QWK = 0.3543, Optimized QWK = 0.462, Optimized thresholds = [0.60763889 0.97314815 2.60802469]
Seed 288380: Mean Train QWK = 0.4735, Mean Validation QWK = 0.3547, Optimized QWK = 0.471, Optimized thresholds = [0.63199267 0.90928927 2.74014775]
Seed 70743: Mean Train QWK = 0.4787, Mean Validation QWK = 0.3643, Optimized QWK = 0.462, Optimized thresholds = [0.58584266 1.01855496 2.66632552]
Seed 887692: Mean Train QWK = 0.4759, Mean Validation QWK = 0.3641, Optimized QWK = 0.462, Optimized thresholds = [0.6028776  0.88912342 2.60138971]
Seed 659334: Mean Train QWK = 0.4709, 

In [28]:
# 0.85 lgbm 0.15 cb: 0.3676 0.470

In [29]:
submission = pd.DataFrame({
        'id': sample['id'],
        'sii': final_preds
    })

submission.to_csv('submission.csv', index=False)

submission.head()

,id,sii
0,00008ff9,0
1,000fd460,0
2,00105258,1
3,00115b9f,0
4,0016bb22,1
